In [383]:
import numpy as np
import cv2
import torch
import kornia
import time

# device = torch.device("cpu")
# device
# 直接设置默认tensor_type为 cuda
torch.set_default_tensor_type('torch.cuda.FloatTensor')
# torch.set_default_tensor_type('torch.FloatTensor')

# 浮雕 emboss

In [395]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
def _emboss(img, intensity = 0.6):
    '''input: 
        img: 3D BGRA tensor (H, W, channel) dtype:torch.float32,range: 0.~255.  (暂未更改成：shape:(batch, channel, H, W))
        The intensity of emboss
        
        output: (H,W,channel) BGRA tensor, range 0.~1.
    '''
    h,w,_ = img.shape
    img = img # Normalize
    b,g,r,a = img.split(1, 2)
    img = torch.cat((r,g,b),2)
#     t0 = time.time() # ---------------------------
#     print(img.shape)
    img = img.cpu().numpy().astype('uint8')
    HSV = cv2.cvtColor(img,cv2.COLOR_RGB2HSV)
    HSV = torch.tensor(HSV)
    HSV = HSV[None,...].permute(0,3,1,2)
#     HSV=kornia.color.rgb_to_hsv(img[None,...].permute(0,3,1,2))
#     t1 = time.time()# ---------------------------
#     delta_t = 1e3*(t1-t0)
#     print(f'rgb_to_hsv time: {delta_t}')
    H,S,V = HSV.split(1,1)

    # Apply the convolution kernel to channel L： 
    p, q, s,t = 0.,0., intensity ,0.  
    # p,q,s control the intensity of the relief in three directions
    # The value of matrix center controls the overall brightness, here sets 1.0
    kernel = np.array([[-q, -s,  t],
                       [-p, 1.0, p],
                       [-t,  s,  q]], dtype=np.float32)
    
    kernel = torch.tensor(kernel,dtype=torch.float32)
    kernel = kernel[None,None,...]
#     t0 = time.time()# ---------------------------
    V = V.type(torch.float32)
    V = torch.conv2d(V, kernel, padding=1)
    V = torch.clamp(V,0,255)
    V = V.type(torch.uint8)
#     t1 = time.time()# ---------------------------
#     delta_t = 1e3*(t1-t0)
#     print(f'conv2d time: {delta_t}')
    
    # Merge each channel
#     HSV = torch.cat((H,S,V),1)
    HSV = torch.cat((H,S,V),1)
#     t0 = time.time()# ---------------------------
    HSV = HSV[-1,...].permute(1,2,0).cpu().numpy()
#     print(HSV.dtype,HSV.shape)
    RGB = cv2.cvtColor(HSV,cv2.COLOR_HSV2RGB)
    RGB = torch.tensor(RGB)
    RGB = RGB[None,...].permute(0,3,1,2)

#     t1 = time.time()# ---------------------------
#     delta_t = 1e3*(t1-t0)
#     print(f'hsv_to_rgb time: {delta_t}')
    RGB = RGB[-1,...].permute(1,2,0)
    
    # The following is to keep the output in BGRA format ,may reduce efficiency
    BGR = RGB
    BGR[:,:,[0,2]] = RGB[:,:,[2,0]]

    return torch.cat((BGR,a),2)

In [396]:
image1 = cv2.imread('gongfuxiongmao.png',cv2.IMREAD_UNCHANGED)
# image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2BGRA)
image1 = torch.tensor(image1,dtype=torch.uint8)

print(f'image1.shape:{image1.shape}')

image1.shape:torch.Size([910, 1000, 4])


In [408]:
torch.set_default_tensor_type('torch.cuda.FloatTensor')
# torch.set_default_tensor_type('torch.FloatTensor')
image1 = cv2.imread('gongfuxiongmao.png',cv2.IMREAD_UNCHANGED)
image1 = torch.tensor(image1,dtype=torch.uint8)

In [398]:
%%timeit
res = _emboss(image1,intensity=1)

13.1 ms ± 295 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


gpu上要比cpu上快10倍.

In [411]:
image3 = _emboss(image1,intensity=1.5)
# plt.figure(figsize=(10,10))
# plt.imshow(image3.cpu());plt.show()

cv2.imshow('',image3.cpu().numpy())
cv2.waitKey(0)
cv2.destroyAllWindows()

In [298]:
image3.shape

torch.Size([910, 1000, 4])

In [378]:
def cv_emboss(img, intensity = 0.6):
    '''input: 
        img: 4D BGRA
        The intensity of emboss
    '''
    import numpy as np
    _,_,_,a = cv2.split(img)

    img = cv2.cvtColor(img,cv2.COLOR_BGRA2RGB)
    t0 = time.time()
    (H,L,S) = cv2.split(cv2.cvtColor(img,cv2.COLOR_RGB2HLS))
    t1 = time.time()
    delta_t = 1e3*(t1-t0)
    print(f'rgb_to_hsv time: {delta_t}')
    # Normalisation：For HSV, Hue range is [0,179], Saturation range is [0,255] and Value range is [0,255].
    H,L,S = H/179., L/255., S/255.
#     H,L,S = H, L, S
    # Apply the convolution kernel to channel L： 
    p, q, s,t = 0.,0., intensity ,0.  
    # p,q,s control the intensity of the relief in three directions
    # The value of matrix center controls the overall brightness, here sets 1.0
    kernel = np.array([[-q, -s,  t],
                       [-p, 1.0, p],
                       [-t,  s,  q]], dtype=np.float32)
    t0 = time.time()
    L = cv2.filter2D(L, -1, kernel)
    t1 = time.time()
    delta_t = 1e3*(t1-t0)
    print(f'fileter2D time: {delta_t}')
    t0 = time.time()
    L = np.clip(L,0,1)
    H,L,S = H*179., L*255., S*255.
    t1 = time.time()
    delta_t = 1e3*(t1-t0)
    print(f'clip time + /255:{delta_t}')
    H,L,S = H.astype('uint8'),L.astype('uint8'),S.astype('uint8')
    #-------------------------------------------------------------------
    # If in the previous step makes the brightness too large, please re-limit the brightness here
    # H,L,S = H, Gamma_trans(L, 255, 1.2), Gamma_trans(S, 255, 1) 
    # H,L,S = H, MaxMinNormal(L, 1, 150), MaxMinNormal(S, 0,255)
    #-------------------------------------------------------------------
    # Merge each channel
    t0 = time.time()
    res = cv2.merge((H,L,S))
    res = cv2.cvtColor(res, cv2.COLOR_HLS2RGB)
    res = cv2.cvtColor(res, cv2.COLOR_RGB2BGRA)

    res[:,:,-1] = a
    t1 = time.time()
    delta_t = 1e3*(t1-t0)
    print(f'hsv_to_rgb time: {delta_t}')
    return res

In [405]:
image1 = cv2.imread('gongfuxiongmao.png',cv2.IMREAD_UNCHANGED)
%time res = cv_emboss(image1, 2.5)

rgb_to_hsv time: 2.9914379119873047
fileter2D time: 4.98652458190918
clip time + /255:15.955924987792969
hsv_to_rgb time: 4.98652458190918
Wall time: 54.8 ms


In [406]:

cv2.imshow('',res)
cv2.waitKey(0)
cv2.destroyAllWindows()